# 处理灾害数据数据，将其入库

数据：“./data-data/灾害数据数据表.xlsx”

## 读取excel


In [ ]:
import pandas as pd 

df=pd.read_excel('./data-data/灾害数据数据表.xlsx',sheet_name='process2',dtype={"DataTimeRange1": "string","DataTimeRange2":"string","DataSpatialRange":"string"})
df.head(3)

## 新建一个ttl文件并写入前缀

In [ ]:
# 创建一个ttl文件，文件名为filename,并向文件写入prefix
def ttl_create(filename, prefix):
    relative_path = "./rdf/"  # 新创建的txt文件的存放路径
    full_path = relative_path + filename + '.ttl'  
    file = open(full_path, 'w')
    file.write(prefix)   #msg也就是下面的Hello world!
    file.close()

name="灾害数据实例"
prefix='''@prefix : <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#> .
@prefix dc: <http://purl.org/dc/elements/1.1/> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schema.org/> .
@base <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology> .

'''

ttl_create(name, prefix)


需要先创建日期实例和矢量地理要素，然后再创建数据实例，并将其关联到相应的日期实例和地理要素

## 创建日期实例
先去除空值，然后将日期字符串根据长度分为年份和年月日两种：
- 针对年份数据，可以直接创建日期实例
- 针对年月日数据，可以依次分割出year、month和day，然后创建实例

In [ ]:
date_from=df['DataTimeRange1'].tolist()
date_to=df['DataTimeRange2'].tolist()
date_list=list(set(date_from+date_to))
print(date_list)

In [ ]:
print(date_list[50])
pd.isnull(date_list[50])

In [ ]:

text=""
for date in date_list:
    
    if pd.isnull(date):
        continue
    date=date[0:10].replace("-","")
    if len(date)>4:     # 创建“年月日”日期实例
        Y=date[0:4]
        M=str(int(date[4:6]))
        D=str(int(date[6:8]))
        date_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{date}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{date}> rdf:type owl:NamedIndividual ,time:Instant ;
time:day {D} ;
time:month {M} ;
time:year {Y} .'''
    else:   # 创建年份实例
        date_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{date}
<http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{date}> rdf:type owl:NamedIndividual ,time:Instant ;
time:year {date} .'''
    text=text+date_text

# 写入文件
file= open("./rdf/灾害数据实例.ttl","a")
file.write(text)
file.close()


## 创建矢量要素

先根据字符串长度排除两个China区域，然后剩下的解析为west、east、south、north这四个坐标，然后拼接为矩形：

polygon.XXXXXX
POLYGON ((west north, west south, east south , east north, west north))


> http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#polygon.AWOFXE
>:polygon.AWOFXE rdf:type owl:NamedIndividual ,
>                         <http://www.opengis.net/ont/sf#Polygon> ;
>                geosparql:asWKT "POLYGON ((70 115, 70 55, 140 55 , 140 15, 70 115))" ;
>                skos:definition "矩形区域：70-140°E, 15-55°N" .


In [ ]:
def create_rectFeature(df):
    text="";
    for index,row in df.iterrows():
        rect_id=row["featureID"]
        rect_str=row["DataSpatialRange"]
        if pd.isnull(rect_id):
            continue
        west,east,south,north=rect_str.split(" ")
        
        fea_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#polygon.{rect_id}
:polygon.{rect_id} rdf:type owl:NamedIndividual , <http://www.opengis.net/ont/sf#Polygon> ;
                geosparql:asWKT "POLYGON (({west} {north}, {west} {south}, {east} {south} , {east} {north}, {west} {north}))" ;
                skos:definition "矩形区域：{west}-{east}°E, {south}-{north}°N" .
        '''

        text=text+fea_text
    return text
        
text = create_rectFeature(df)    
file = open("./rdf/灾害数据实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()

## 创建数据实例


In [ ]:
def create_dataIndividual(df):
    text="";
    for index,row in df.iterrows():
        identifier=row["DataIdentifier"]
        name=row["DataName"]
        description=row["DataDescription"]
        subclass=row["DataCategory"].replace(";",", :")
        begin=row["DataTimeRange1"]
        end=row["DataTimeRange2"]
        featureID=row["featureID"]
        dformat=row["DataFormat"]+"DataFormat"
        fileformat=row["fileFormat"]
        source=row["DataSource"]
        address=row["DataAddress"]
        srs=row["SRS-SRID"]
        s_resolution=row["SpatialResolution"]
        t_resolution=row["frequency"]
        bandNum=row["bandNum"]
        colNum=row["colNum"]
        rowNum=row["rowNum"]
        unit=row["unit"]
        otherInfo=["otherInfo"]

        
        data_text=f'''
###  http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{identifier}
:{identifier} rdf:type owl:NamedIndividual , :{subclass} ;
:hasFormat :{dformat} ;
:dataAddress "{address}" ;
:dataDescription "{description}" ;
:dataIdentifier "{identifier}" ;
:dataName "{name}" ;
:dataSource "{source}" ;
:dataformat "{fileformat}" ;
rdfs:label "{name}" .
'''

        if not pd.isnull(begin):
            if len(begin)>4:
                begin=begin[0:10].replace("-","")
            data_text=data_text+f'''
:{identifier} :begin <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{begin}> .'''

        if not pd.isnull(end):
            if len(end)>4:
                end=end[0:10].replace("-","")
            data_text=data_text+f'''
:{identifier} :end <http://www.semanticweb.org/wen/ontologies/2022/8/FloodDisasterEventOntology#{end}> .'''
        
        if not pd.isnull(featureID):
            data_text=data_text+f'''
:{identifier} :geoFeature :polygon.{featureID} .'''

        if not pd.isnull(bandNum):
            data_text=data_text+f'''
:{identifier} :bandNumber {bandNum} .'''

        if not pd.isnull(colNum):
            data_text=data_text+f'''
:{identifier} :columnNumber {colNum} .'''

        if not pd.isnull(rowNum):
            data_text=data_text+f'''
:{identifier} :rowNumber {rowNum} .'''

        if not pd.isnull(unit):
            data_text=data_text+f'''
:{identifier} :statisticUnit "{unit}" .'''

        if not pd.isnull(srs):
            data_text=data_text+f'''
:{identifier} :spatialReference "{srs}" .'''

        if not pd.isnull(s_resolution):
            data_text=data_text+f'''
:{identifier} :spatialResolution "{s_resolution}" .'''
        if not pd.isnull(t_resolution):
            data_text=data_text+f'''
:{identifier} :temporalResolution "{t_resolution}" .'''
        if not pd.isnull(otherInfo):
            data_text=data_text+f'''
:{identifier} :otherInfo "{otherInfo}" .'''

        text=text+data_text
    return text
        
text = create_dataIndividual(df)    
file = open("./rdf/灾害数据实例.ttl", 'a',encoding="utf-8")
file.write(text)
file.close()